In [1]:
#Import the Librabies

%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import plotly as py
import plotly.figure_factory as ff
import ipywidgets as widgets
import plotly.graph_objs as go
#import pywt
import scipy.signal
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
py.offline.init_notebook_mode(connected = True)
from plotly import tools
#from obspy.signal.detrend import polynomial
from hrv_analysis.extract_features import _create_interpolation_time, _create_time_info
from scipy import interpolate

In [2]:
srate = 700
window_length = 32*srate
seconds_per_window = window_length / srate

In [3]:
path = "C:/Users/ee19s/Desktop/HR/PPG_FieldStudy/S14/S14.pkl"
subpath_activity = pd.read_csv("C:/Users/ee19s/Desktop/HR/PPG_FieldStudy/S14/S14_activity.csv")
subpath_activity = subpath_activity.rename(columns = {'# SUBJECT_ID':'subject_id'}) 
subpath_activity['subject_id'] = subpath_activity.iloc[:,0].astype('category') 
subpath_activity['activity_id'] = subpath_activity.subject_id.cat.codes
start_time = subpath_activity.iloc[: , 1].values 

In [4]:
for index in range(1,len(subpath_activity)):
            if index == 1:
                annotation_per_window = [subpath_activity.iloc[index-1,2] for i in range(int(round(subpath_activity.iloc[index,1] / seconds_per_window)))]
                prev = round(subpath_activity.iloc[index,1] / seconds_per_window) * seconds_per_window 
            else:
                annotation_per_window += [subpath_activity.iloc[index-1,2] for i in range(int(round((subpath_activity.iloc[index,1] - prev) / seconds_per_window)))]
                prev = round(subpath_activity.iloc[index,1] / seconds_per_window) * seconds_per_window

In [5]:
subpath_activity

,subject_id,S14,activity_id
0,# NO_ACTIVITY,0,5
1,# BASELINE,62,0
2,# NO_ACTIVITY,671,5
3,# STAIRS,814,7
4,# NO_ACTIVITY,1271,5
5,# SOCCER,1400,6
6,# NO_ACTIVITY,1726,5
7,# CYCLING,2210,2
8,# NO_ACTIVITY,2713,5
9,# DRIVING,3341,3


In [6]:
with open (path , 'rb') as f:
    data_dict = pickle.load(f , encoding= 'bytes')
acc_x = []
acc_y = []
acc_z = []
srate = 700
ecg_data = data_dict[b'signal'][b'chest'][b'ECG']
acc_data = data_dict[b'signal'][b'chest'][b'ACC']
respiratory_data = data_dict[b'signal'][b'chest'][b'Resp']
rpeaks = data_dict[b'rpeaks']
ecg_data = ecg_data.flatten()
raw_resp = respiratory_data.flatten()
for item in acc_data:
    acc_x.append(item[0])
    acc_y.append(item[1])
    acc_z.append(item[2])
acc_x_axis = np.array(acc_x)
acc_y_axis = np.array(acc_y)
acc_z_axis = np.array(acc_z)
acc_data = acc_y_axis + acc_z_axis
#start_time = dataset.iloc[: , 1].values


In [7]:
ecg = []
resp = []
acc = []

window_length = 32*srate
len_parameter = int(np.round(len(ecg_data)/window_length))
peak_loc = [np.array([]) for i in range(len_parameter)]
peak_amp = [np.array([]) for i in range(len_parameter)]
for i in range(len_parameter):
    ecg.append(ecg_data[i*window_length : (i+1)*window_length])
    resp.append(raw_resp[i*window_length : (i+1)*window_length])
    acc.append(acc_data[i*window_length : (i+1)*window_length])
    for item in rpeaks:
        if item >= i*window_length and item <= (i+1)*window_length:
            sub = i*window_length
            item1 = ecg_data[item]
            peak_loc[i] = np.append(peak_loc[i] , item-sub)
            peak_amp[i] = np.append(peak_amp[i] , item1)
            


In [8]:
#fbpB , fbpA = band_pass(0.1,0.9,8)
#fkern_lp_B ,fkern_lp_A = cheby_lp(6,25,1)              
#fkern_hp_B , fkern_hp_A = cheby_hp(4,30,0.1)
frange = np.array([0.1 , 0.9])
fbpB,fbpA = scipy.signal.butter(8,np.array(frange)/2,btype='bandpass')
fkern_lp_B ,fkern_lp_A = scipy.signal.cheby2(6 , 25 , 1/(srate/2) , btype='lowpass')
fkern_hp_B , fkern_hp_A  = scipy.signal.cheby2(4 , 30 , 0.1/(srate/2) , btype='highpass')

flp_respB , flp_respA = scipy.signal.cheby2(5 , 30 , 0.7/(srate/2) , btype='lowpass')
fhp_respB , fhp_respA  = scipy.signal.cheby2(4 , 20 , 0.1/(srate/2) , btype='highpass')


In [9]:
final_resp = []
for item in resp:
    lp_filt_sig = scipy.signal.filtfilt(flp_respB , flp_respA , item)
    final_resp.append(scipy.signal.filtfilt(fhp_respB,fhp_respA ,lp_filt_sig ))



In [10]:
def edr_adr_extraction(acc , rpeaks , rpeak_amplitudes,ds_factor = 10):
    '''
    inputs -- acc - Accelerometer signal extracted from dictionary returned by PPG_Dalia_data_extraction()
              rpeaks - R peak indices obtained from dictionary returned by PPG_Dalia_data_extraction()
              rpeak_amplitudes - R peak amplitudes obtained from dictionary returned by PPG_Dalia_data_extraction()
    
    outputs -- Function returns edr signals by HRV and RPEAK amplitude variations and ADR signal from accelerometer.

    Description -- Function takes the ACC, RPEAKS, Rpeak amplitudes for a particular subject and then calculate
                  the respiratory signal based of HRV, rpeak amplitude variations and using adaptive filtering
                  for accelerometer data. 
    '''
    final_edr_hrv = []
    final_edr_rpeak = []
    final_adr = []
    #-----------------------RESPIRATORY SIGNAL BY HRV------------------------------------
    # interpolate the rr interval using cubic spline interpolation and filter between 
    # 0.1Hz - 1Hz to obtain final edr
    for item in rpeaks:
        rr_interval = (np.diff(item)/srate)*1000
        index = np.where(rr_interval == 0)
        rr_interval = np.delete(rr_interval , index)
        rr_times = _create_time_info(list(rr_interval))
        funct = interpolate.interp1d(x=rr_times, y=list(rr_interval), kind='cubic')
        timestamps_interpolation = _create_interpolation_time(rr_times, 4)
        interpolated_signal = funct(timestamps_interpolation)
        #time_stamp_hrv = np.arange(0,len(rr_interval))
        #time_stamp_hrv = np.cumsum(rr_interval/srate)
        #time_stamp_hrv = time_stamp_hrv - time_stamp_hrv[0]
        #time_interp_hrv = np.arange(time_stamp_hrv[0] , time_stamp_hrv[-1] , 1/4)
        #interpolated_signal = scipy.interpolate.griddata(time_stamp_hrv , rr_interval , time_interp_hrv , method='cubic')
        interpolated_signal = (interpolated_signal - np.mean(interpolated_signal))/np.std(interpolated_signal)
        final_edr_hrv.append(scipy.signal.filtfilt(fbpB , fbpA , interpolated_signal))
        
    #---------------------RESPIRATORY SIGNAL BY RPEAKS-----------------------------------
    # interpolate the rpeak amplitudes using cubic spline interpolation and filter between 
    # 0.1Hz - 1Hz to obtain final edr
    i = 0
    for item in rpeak_amplitudes:
        #import pdb;pdb.set_trace() 
        rr_interval = (np.diff(rpeaks[i])/srate)*1000
        index = np.where(rr_interval == 0)
        rr_interval = np.delete(rr_interval , index)
        item = np.delete(item,index)
        rr_times = _create_time_info(list(rr_interval))
        funct = interpolate.interp1d(x=rr_times, y=list(item[1:]), kind='cubic')
        timestamps_interpolation = _create_interpolation_time(rr_times, 4)
        interpolated_signal_rp = funct(timestamps_interpolation)
        #time_stamp_rpeak = np.arange(0 , len(item))
        #time_interp_rpeak = np.arange(time_stamp_rpeak[0] , time_stamp_rpeak[-1] , 1/4)
        #interpolated_signal_rp = scipy.interpolate.griddata(time_stamp_rpeak , item , time_interp_rpeak ,method='cubic' )
        interpolated_signal_rp = (interpolated_signal_rp - np.mean(interpolated_signal_rp))/np.std(interpolated_signal_rp)
        final_edr_rpeak.append(scipy.signal.filtfilt(fbpB,fbpA , interpolated_signal_rp))
        i+=1
    #-------------------------RESPIRATORY SIGNAL BY ACCELEROMETER-------------------------
    # calculate the fft of accelerometer data and then select the spectrum between
    # the frequency range of 0.1Hz - 1Hz the frequency correspond to the maximum
    # power will be taken as central frequency and then that will decide the 
    # lower cut off frequency or upper cuttoff frequency of the filter to obtain
    # the respiratory signal.
    for item in acc:
        lp_filt_sig = scipy.signal.filtfilt(fkern_lp_B , fkern_lp_A , item)
        hp_filt_sig = scipy.signal.filtfilt(fkern_hp_B , fkern_hp_A , lp_filt_sig)
        spectrum = np.absolute(scipy.fft.fft(hp_filt_sig)**2)
        freq = scipy.fft.fftfreq(len(spectrum) , d= 1/srate)
        rel_freq = freq[3:33]
        rel_spectrum = spectrum[3:33]
        max_freq = rel_freq[np.argmax(rel_spectrum)]
        lower_cut_freq = max(0.1 , max_freq-0.4)
        upper_cut_freq = max_freq + 0.4
        flpB ,flpA = scipy.signal.cheby2(5,30,upper_cut_freq/(srate/2) , btype='lowpass')
        fhpB , fhpA = scipy.signal.cheby2(4,30, lower_cut_freq/(srate/2) , btype='highpass')
        lp_filt_acc = scipy.signal.filtfilt(flpB, flpA , hp_filt_sig)
        final_signal = scipy.signal.filtfilt(fhpB , fhpA, lp_filt_acc)
        final_adr.append(scipy.signal.decimate(final_signal , ds_factor))
    return final_edr_hrv , final_edr_rpeak , final_adr


In [11]:
edr_hrv , edr_rpeak , adr = edr_adr_extraction(acc ,peak_loc,peak_amp )  

In [12]:
print(len(edr_hrv[0]))

124


In [13]:
def extremas_extraction(signal):
    avg_breath_duration = np.array([])
    extrema_relevent = []
    for item in signal:
        amplitude = np.array([])
        pos_peaks , _ = scipy.signal.find_peaks(item , height = [-300,300])
        neg_peaks , _ = scipy.signal.find_peaks(-1*item , height = [-300 , 300])
        extremas = np.concatenate((pos_peaks , neg_peaks))
        extremas = np.sort(extremas)
        for i in range(len(extremas)):
            amplitude = np.append(amplitude , item[int(extremas[i])])
        amplitude_diff = np.abs(np.diff(amplitude))
        q3 = np.percentile(amplitude_diff , 75)
        threshold = 0.3*q3
        eliminate_pairs_of_extrema = 1
        while(eliminate_pairs_of_extrema):
            amps = np.array([])
            if len(extremas)<3:
                eliminate_pairs_of_extrema = 0
                continue
            for i in range(len(extremas)):
                amps = np.append(amps , item[int(extremas[i])])
            amp_diff = np.abs(np.diff(amps)) 
            min_amp_diff , index = min(amp_diff) , np.argmin(amp_diff)
            #print(min_amp_diff)
            if min_amp_diff > threshold:
                eliminate_pairs_of_extrema = 0
                #extrema_relevent = extremas
            else:
                extremas = np.concatenate((extremas[0:index] , extremas[index+2 :]))
                #amplitude_diff = np.delete(amplitude_diff , index)
        if item[int(extremas[0])] < item[int(extremas[1])]:
            extremas = extremas[1:]
        if item[int(extremas[-1])] < item[int(extremas[-2])]:
            extremas = extremas[:-1]
        no_of_breaths = (len(extremas)-1)/2
        breath_duration = extremas[-1] - extremas[0]
        avg_breath_duration = np.append(avg_breath_duration , breath_duration/no_of_breaths)
        extrema_relevent.append(extremas)
    return avg_breath_duration , extrema_relevent

In [14]:
duration_hrv,extrema_hrv = extremas_extraction(edr_hrv)
duration_rpeak,extrema_rpeak = extremas_extraction(edr_rpeak)
duration_adr,extrema_acc = extremas_extraction(adr)
duration_resp,extrema_resp = extremas_extraction(final_resp)



In [15]:
dura_hrv = np.diff(extrema_hrv[7][::2])
dura_rpeak = np.diff(extrema_rpeak[7][::2])
dura_adr = np.diff(extrema_acc[7][::2])
dura_resp = np.diff(extrema_resp[7][::2])

In [16]:
print(dura_hrv)

[15 16 15 13 15 13 14 13]


In [17]:
br_hrv = 60*4/dura_hrv
br_rpeak = 60*4/dura_rpeak
br_adr = 60*70/dura_adr
br_resp = 60*700/dura_resp
print(br_hrv)
print(br_rpeak)
print(br_adr)
print(br_resp)

print(np.mean(br_hrv))
print(np.mean(br_rpeak))
print(np.mean(br_adr))
print(np.mean(br_resp))

[16.         15.         16.         18.46153846 16.         18.46153846
 17.14285714 18.46153846]
[18.46153846  7.74193548  9.23076923 24.         14.11764706]
[22.58064516  8.84210526 13.125      17.21311475 18.66666667 15.78947368
 17.28395062]
[17.06623324 14.88833747 15.02683363 18.40490798 16.58767773 17.21311475
 18.04123711 16.74641148]
16.940934065934066
14.710378047000438
16.21442230667253
16.74684417379012


In [18]:
total_win_hrv = 32*4
total_win_rpeak = 32*4
total_win_acc = int(32*(srate/10))
total_win_resp = 32*srate

In [19]:
no_of_samples_in_edr = 32*4
no_of_samples_in_adr = int(32*(srate/10))
no_of_samples_in_adr_resp = 32*srate
x_edr = np.linspace(start = 0,stop = no_of_samples_in_edr, num = no_of_samples_in_edr)
x_adr = np.linspace(start = 0,stop = no_of_samples_in_adr, num = no_of_samples_in_adr)
x_resp = np.linspace(start = 0,stop = no_of_samples_in_adr_resp, num = no_of_samples_in_adr_resp)


In [20]:
annotations = np.array(annotation_per_window)
no_activity_index = np.where(annotations == 5)
baseline_index = np.where(annotations == 0)
stairs_index = np.where(annotations == 7)
soccer_index = np.where(annotations == 6)
cycling_index = np.where(annotations == 2)
driving_index = np.where(annotations == 3)
lunch_index = np.where(annotations == 4)
walking_index = np.where(annotations == 8)
working_index = np.where(annotations == 9)
clean_baseline_index = np.where(annotations == 1)


In [21]:
walking_index

(array([202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214,
        215, 216, 217, 218, 219, 220], dtype=int64),)

In [22]:
breath_hrv = 60/(duration_hrv/4)
breath_rpeak = 60/(duration_rpeak/4)
breath_adr = 60/(duration_adr/(srate/10))
breath_resp = 60/(duration_resp/srate)

In [23]:
print(breath_hrv[205])
print(breath_rpeak[205])
print(breath_adr[206])
print(breath_resp[206])
print("-----------------------------")
    


17.2972972972973
25.945945945945947
21.010505252626313
17.91044776119403
-----------------------------


In [24]:
layout_edr_hrv = go.Layout(
    title = "Plotting EDR Waveforms for ECG by HRV",
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples'
    )
)

layout_edr_rpeak = go.Layout(
    title = "Plotting EDR Waveforms for ECG by rpeak",
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples'
    )
)

layout_adr = go.Layout(
    title = "Plotting ADR waveforms for ACC",
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples'
    )
)

layout_resp = go.Layout(
    title = "Plotting resp signal waveforms for ref. resp. signal",
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples'
    )
)

def update_plot(signals):
    data = []
    # Reference ECG trace
    trace_edr_hrv = go.Scatter(
             x = x_edr,
             y = edr_hrv[signals], 
             mode = 'lines',
             name = 'edr by hrv',
             line = dict(
                 shape = 'spline'
             )
        )
    trace_hrv_extremas = go.Scatter(
        x = extrema_hrv[signals],
        y = edr_hrv[signals][extrema_hrv[signals]], 
        mode = 'markers',
        name = 'extremas',
        marker = dict(
                 size = 8,
                 color = 'red',
                 symbol = 'cross'
                 
             )
        )
    trace_edr_rpeak = go.Scatter(
             x = x_edr,
             y = edr_rpeak[signals], 
             mode = 'lines',
             name = 'edr by rpeak',
             line = dict(
                 shape = 'spline'
             )
        )
    
    trace_rpeak_extremas = go.Scatter(
        x = extrema_rpeak[signals],
        y = edr_rpeak[signals][extrema_rpeak[signals]], 
        mode = 'markers',
        name = 'extremas',
        marker = dict(
                 size = 8,
                 color = 'red',
                 symbol = 'cross'
                 
             )
        )
    trace_adr = go.Scatter(
             x = x_adr,
             y = adr[signals], 
             mode = 'lines',
             name = 'adr from acc',
             line = dict(
                 shape = 'spline'
             )
        )
    trace_adr_extremas = go.Scatter(
        x = extrema_acc[signals],
        y = adr[signals][extrema_acc[signals]], 
        mode = 'markers',
        name = 'extremas',
        marker = dict(
                 size = 8,
                 color = 'red',
                 symbol = 'cross'
                 
             )
        )
    trace_resp = go.Scatter(
             x = x_resp,
             y = final_resp[signals], 
             mode = 'lines',
             name = 'reference resp signal',
             line = dict(
                 shape = 'spline'
             )
        )
    trace_resp_extremas = go.Scatter(
        x = extrema_resp[signals],
        y = final_resp[signals][extrema_resp[signals]], 
        mode = 'markers',
        name = 'extremas',
        marker = dict(
                 size = 8,
                 color = 'red',
                 symbol = 'cross'
                 
             )
        )
    fig_edr_hrv = go.Figure(data = [trace_edr_hrv , trace_hrv_extremas],layout = layout_edr_hrv)
    py.offline.iplot(fig_edr_hrv)
    fig_edr_rpeak = go.Figure(data = [trace_edr_rpeak , trace_rpeak_extremas],layout = layout_edr_rpeak)
    py.offline.iplot(fig_edr_rpeak)
    fig_adr = go.Figure(data = [trace_adr , trace_adr_extremas],layout = layout_adr)
    py.offline.iplot(fig_adr)
    fig_resp = go.Figure(data = [trace_resp , trace_resp_extremas],layout = layout_resp)
    py.offline.iplot(fig_resp)
signals_edr_hrv = widgets.IntSlider(min = 0,max = len(edr_hrv), value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_edr_hrv)

signals_edr_rpeak = widgets.IntSlider(min = 0,max = len(edr_rpeak), value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_edr_rpeak)
signals_adr = widgets.IntSlider(min = 0,max = len(adr), value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_adr)
signals_resp = widgets.IntSlider(min = 0,max = len(final_resp), value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_resp)

interactive(children=(IntSlider(value=0, description='Record_no:', max=280), Output()), _dom_classes=('widget-…